In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

from scipy import stats
from scipy.ndimage import gaussian_filter
import pingouin as pg

import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import date
import time

import random

from joblib import Parallel, delayed

import h5py

In [16]:
mother_path = Path('D:/Multi-modal project/')

cell_path = mother_path/'analysis'/'result'/'zFR export'/'13-Jun-2023'
cell_list = os.listdir(cell_path)

df = pd.read_csv(mother_path / 'analysis' / 'result' / '2. Unit profiler' / '14-May-2021' / 'Unit_profiler.csv')

In [25]:
df

,Key,RatID,Session,TT,Cluster,Task_session,Task,Region,Subregion,Layer,...,Spike_Height(uv),MeanFR(base),MeanFR(sample),MeanFR(choice),Mode_latency(ms),Cell_CAT,IsolationDistance,L-ratio,withinREFRACPortion,LogISIPEAKTIME(ms)
0,1,600,1,3,1,1,Crossmodal,TeV,na,deep,...,102.33,0.11,0.01,2868.28,137,regular,6.71,1.03,0.000,120.23
1,2,600,1,3,2,1,Crossmodal,TeV,na,deep,...,96.88,0.01,0,1771.72,116,regular,3.17,4.08,0.000,234.42
2,3,600,1,4,1,1,Crossmodal,TeV,na,deep,...,148.87,2.18,4.51,19386.69,80,regular,11.30,0.63,0.016,102.33
3,4,600,1,4,2,1,Crossmodal,TeV,na,deep,...,90.81,4.14,5.26,21284.89,80,regular,8.05,1.10,0.109,165.96
4,5,600,1,4,3,1,Crossmodal,TeV,na,deep,...,91.81,1.75,1.58,13358.71,496,regular,4.85,3.44,0.059,562.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524,2525,699,11,17,2,2,NovelComb,PER,A36,deep,...,124.58,3.94,3.51,24317.62,98,regular,12.69,0.52,0.033,100.00
2525,2526,699,11,17,3,2,NovelComb,PER,A36,deep,...,119.46,2.84,3.49,25422.72,134,regular,12.96,0.48,0.035,151.36
2526,2527,699,11,19,1,2,NovelComb,Visual,secondary,deep,...,197.15,26.7,54.04,229287.66,11,bursting,46.32,0.25,0.126,12.59
2527,2528,699,11,24,1,2,NovelComb,Visual,secondary,deep,...,309.48,13.2,16.06,98462.64,54,regular,43.71,0.24,0.001,54.95


In [28]:
df[df['Key']==cell_id]

,Key,RatID,Session,TT,Cluster,Task_session,Task,Region,Subregion,Layer,...,Spike_Height(uv),MeanFR(base),MeanFR(sample),MeanFR(choice),Mode_latency(ms),Cell_CAT,IsolationDistance,L-ratio,withinREFRACPortion,LogISIPEAKTIME(ms)
2408,2409,699,6,23,1,6,Crossmodal,Visual,secondary,superficial,...,236.65,9.88,16,73575.02,16,bursting,27.28,0.22,0.011,31.62


In [35]:
df.columns

Index(['Key', ' RatID', ' Session', ' TT', ' Cluster', ' Task_session',
       ' Task', ' Region', ' Subregion', ' Layer', ' Bregma',
       ' Spike_Width(ms)', ' Spike_Height(uv)', ' MeanFR(base)',
       ' MeanFR(sample)', ' MeanFR(choice)', ' Mode_latency(ms)', ' Cell_CAT',
       ' IsolationDistance', ' L-ratio', ' withinREFRACPortion',
       ' LogISIPEAKTIME(ms)'],
      dtype='object')

In [30]:
cell_id

6

In [39]:
df[' Cell_CAT'].unique()

array([' regular', ' bursting', ' unclassified'], dtype=object)

In [46]:
cat_list = np.zeros(3)
for c in cell_list:
    cell_info = c.split('-')
    
    cell_id = int(cell_info[0])
    ss_type = cell_info[4]
    region = cell_info[5]
    
    if (ss_type!='Crossmodal') or (region!='PER'):
        continue
    
    cell_cat = df.loc[df['Key']==cell_id,' Cell_CAT'].item
    if cell_cat == ' regular':
        cat_list[0] += 1
    elif cell_cat == ' bursting':
        cat_list[1] += 1
    elif cell_cat == ' unclassified':
        cat_list[2] += 1